In [1]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile
import zipfile
import subprocess
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

Tensorflow version: 1.14.0


In [2]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(
    PROJ_DIR, 'AFHQ-DOG')

In [3]:
RAW_IMGS_DIR_ = f'{PROJ_DIR}/datasets/AFHQ_images_raw/afhq/train/dog'

In [4]:
raw_imgs = glob(f'{RAW_IMGS_DIR_}/*')
print(len(raw_imgs))

4739


In [8]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

100%|██████████| 4739/4739 [02:54<00:00, 27.22it/s]


In [5]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 4739, Resized: 4739


In [10]:
# DeepFAMS.preprocessing.tf_record_exporter(tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/AFHQ-DOG_resized"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/AFHQ-DOG_custom"
Added 4739 images.                      


In [6]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada/train.py \
--outdir={TRAIN_RUNS_DIR} \
--gpus=2 \
--data={DATA_CUSTOM_DIR} \
--snap=1 \
--kimg=1''', PROJ_DIR)



Training options:

{

  "G_args": {

    "func_name": "training.networks.G_main",

    "fmap_base": 8192,

    "fmap_max": 512,

    "mapping_layers": 2,

    "num_fp16_res": 4,

    "conv_clamp": 256

  },

  "D_args": {

    "func_name": "training.networks.D_main",

    "mbstd_group_size": 4,

    "fmap_base": 8192,

    "fmap_max": 512,

    "num_fp16_res": 4,

    "conv_clamp": 256

  },

  "G_opt_args": {

    "beta1": 0.0,

    "beta2": 0.99,

    "learning_rate": 0.0025

  },

  "D_opt_args": {

    "beta1": 0.0,

    "beta2": 0.99,

    "learning_rate": 0.0025

  },

  "loss_args": {

    "func_name": "training.loss.stylegan2",

    "r1_gamma": 0.4096

  },

  "augment_args": {

    "class_name": "training.augment.AdaptiveAugment",

    "tune_heuristic": "rt",

    "tune_target": 0.6,

    "apply_func": "training.augment.augment_pipeline",

    "apply_args": {

      "xflip": 1,

      "rotate90": 1,

      "xint": 1,

      "scale": 1,

      "rotate": 1,

      "aniso": 1,


network-snapshot-000000        time 2m 24s       fid50k_full 308.9665

tick 1     kimg 1.0      time 5m 53s       sec/tick 13.1    sec/kimg 14.64   maintenance 170.7  gpumem 7.1   augment 0.001

Evaluating metrics...

network-snapshot-000001        time 2m 25s       fid50k_full 348.3302



Exiting...




In [7]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ADA_Project/training_runs/AFHQ-DOG_training-runs/00000-AFHQ-DOG_custom-auto2-kimg1/network-snapshot-000001.pkl


In [ ]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 10,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [ ]:
DeepFAMS.utils.execute('nvidia-smi')

In [ ]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)